In [ ]:
###final code
import os
import io
import csv
from google.cloud import videointelligence

def detect_labels_from_file(input_path, output_path):
    video_client = videointelligence.VideoIntelligenceServiceClient()
    features = [videointelligence.Feature.LABEL_DETECTION]

    with open(output_path, 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['Video File', 'Type', 'Description', 'Category', 'Segment', 'Confidence']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

        for file_name in os.listdir(input_path):
            if file_name.endswith('.mp4'):
                video_file_path = os.path.join(input_path, file_name)
                print(f"\nProcessing video file: {video_file_path}")

                with io.open(video_file_path, "rb") as movie:
                    input_content = movie.read()

                operation = video_client.annotate_video(
                    request={"features": features, "input_content": input_content}
                )
                print("Finished processing.")

                result = operation.result(timeout=180)

                # Process video/segment level label annotations
                for segment_label in result.annotation_results[0].segment_label_annotations:
                    for segment in segment_label.segments:
                        start_time = (
                            segment.segment.start_time_offset.seconds
                            + segment.segment.start_time_offset.microseconds / 1e6
                        )
                        end_time = (
                            segment.segment.end_time_offset.seconds
                            + segment.segment.end_time_offset.microseconds / 1e6
                        )
                        positions = "{}s to {}s".format(start_time, end_time)
                        confidence = segment.confidence
                        writer.writerow({
                            'Video File': file_name,
                            'Type': 'Video/segment',
                            'Description': segment_label.entity.description,
                            'Category': segment_label.category_entities[0].description if segment_label.category_entities else '',
                            'Segment':  positions,
                            'Confidence': confidence
                        })

                # Process shot level label annotations
                for shot_label in result.annotation_results[0].shot_label_annotations:
                    for shot in shot_label.segments:
                        start_time = (
                            shot.segment.start_time_offset.seconds
                            + shot.segment.start_time_offset.microseconds / 1e6
                        )
                        end_time = (
                            shot.segment.end_time_offset.seconds
                            + shot.segment.end_time_offset.microseconds / 1e6
                        )
                        positions = "{}s to {}s".format(start_time, end_time)
                        confidence = shot.confidence
                        writer.writerow({
                            'Video File': file_name,
                            'Type': 'Shot',
                            'Description': shot_label.entity.description,
                            'Category': shot_label.category_entities[0].description if shot_label.category_entities else '',
                            'Segment':  positions,
                            'Confidence': confidence
                        })

                # Process frame level label annotations
                for frame_label in result.annotation_results[0].frame_label_annotations:
                    for frame in frame_label.frames:
                        time_offset = frame.time_offset.seconds + frame.time_offset.microseconds / 1e6
                        confidence = frame.confidence
                        writer.writerow({
                            'Video File': file_name,
                            'Type': 'Frame',
                            'Description': frame_label.entity.description,
                            'Category': frame_label.category_entities[0].description if frame_label.category_entities else '',
                            'Segment': f"Frame {time_offset}s",
                            'Confidence': confidence
                        })

# Path to the folder containing the video files
input_folder_path = r"D:\PhD_Socialmediadata\Tiktok_Jordaan\videos\11"

# Path to save the CSV file
output_csv_path = os.path.join(input_folder_path, 'video_labels.csv')

# Call the function to detect labels for all video files in the folder and save the results to a CSV file
detect_labels_from_file(input_folder_path, output_csv_path)